In [1]:
import pandas as pd

df = pd.read_csv("abono.csv")

In [2]:
df.head()

,Unnamed: 0,Ano,Mes,Publicacao,Unnamed: 0.1,Tipo do Ato,Nome do Servidor,Matrícula,Cargo Efetivo,Classe,Padrão,Quadro pessoal permanente ou Suplementar,Fundamento Legal do abono de permanência,Órgão,Processo GDF/SEI,Vigencia,Matricula SIAPE,text
0,0,2013,7,DODF Nº 145 16-07-2013,0,Abono de Permanência,NaN,42.825-6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abono de Permanencia aoservidor HAROLDO DA SIL...
1,1,2013,7,DODF Nº 154 29-07-2013,0,Abono de Permanência,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"abono de permanencia e,posteriormente, convert..."
2,2,2013,7,DODF Nº 143 12-07-2013,0,Abono de Permanência,Escrivao de Policia,SIGRH,NaN,NaN,NaN,NaN,artigo 1o,NaN,NaN,NaN,NaN,Abono de Permanencia a:FRANCISCO \nAURELIO DIA...
3,3,2013,7,DODF Nº 152 25-07-2013,0,Abono de Permanência,\nVALDIVINO RODRIGUES,102250-4,Natureza \nEspecial,NaN,NaN,NaN,NaN,NaN,NaN,12 de maio \nde 2012,NaN,Abono de Permanencia aFRANCISCO \nVALDIVINO RO...
4,4,2013,7,DODF Nº 141 10-07-2013,0,Abono de Permanência,ANTONIO CARLOS DIAS ALMEIDA,19.913-3,NaN,Especial,V,Quadro de Pessoal do Distrito Federal,NaN,NaN,NaN,NaN,NaN,Abono de Permanencia a\nANTONIO CARLOS DIAS AL...


In [3]:
df.columns

Index(['Unnamed: 0', 'Ano', 'Mes', 'Publicacao', 'Unnamed: 0.1', 'Tipo do Ato',
       'Nome do Servidor', 'Matrícula', 'Cargo Efetivo', 'Classe', 'Padrão',
       'Quadro pessoal permanente ou Suplementar',
       'Fundamento Legal do abono de permanência', 'Órgão', 'Processo GDF/SEI',
       'Vigencia', 'Matricula SIAPE', 'text'],
      dtype='object')

In [4]:
df = df.drop(['Unnamed: 0', 'Ano', 'Mes', 'Publicacao', 'Unnamed: 0.1', 'Tipo do Ato'], axis=1)
df.columns

Index(['Nome do Servidor', 'Matrícula', 'Cargo Efetivo', 'Classe', 'Padrão',
       'Quadro pessoal permanente ou Suplementar',
       'Fundamento Legal do abono de permanência', 'Órgão', 'Processo GDF/SEI',
       'Vigencia', 'Matricula SIAPE', 'text'],
      dtype='object')

In [6]:
df = df.rename(columns={"Matrícula":"matricula", "Padrão":"padrao", "Fundamento Legal do abono de permanência":"fundamento legal do abono de permanencia", "Órgão":"orgao"})
df.columns

Index(['Nome do Servidor', 'matricula', 'Cargo Efetivo', 'Classe', 'padrao',
       'Quadro pessoal permanente ou Suplementar',
       'fundamento legal do abono de permanencia', 'orgao', 'Processo GDF/SEI',
       'Vigencia', 'Matricula SIAPE', 'text'],
      dtype='object')

In [7]:
def data_setup(df):
    for row in range(len(df)):
        for col in df.columns:
            aux = df.iloc[row][col]
            if pd.notna(aux) and aux:
                if type(aux)!=str:
                    aux = str(aux)
                if col == 'text':
                    aux = aux.replace(',', ' , ').replace(';', ' ; ').replace(':', ' : ').replace('. ', ' . ').replace('\n', ' ')
                    if aux[len(aux)-2:] == '. ':
                        aux = aux[:len(aux)-2] + " ."
                else:
                    if aux[len(aux)-1] == '.':
                        aux = aux[:len(aux)-1]
                    aux = aux.replace(',', ' , ').replace(':', ' : ').replace(';', ' ; ').replace('. ', ' . ').replace('\n', ' ')

                df.iloc[row][col] = aux
    return df

def preprocess(df):
    x = []
    y = []
    for row in range(len(df)):
        temp_x = df.iloc[row]['text'].split()
        temp_y = ['O' for i in range(len(temp_x))]
        for col in df.columns:
            if col == 'text':
                continue
            if pd.notna(df.iloc[row][col]):
                feature = df.iloc[row][col].split()
                if not feature:
                    continue
                for i in range(len(temp_x)-len(feature)+1):
                    if feature == temp_x[i:i+len(feature)]:
                        if i >= len(temp_y):
                            print("row:", row)
                        temp_y[i] = "B-"+col
                        for j in range(1, len(feature)):
                            temp_y[i+j] = "I-"+col
        x.append(temp_x)
        y.append(temp_y)
    return x, y

# mini_df = df[:1]
dataframer = data_setup(df)
data_x, data_y = preprocess(dataframer)

In [8]:
for i in range(len(data_x)):
    for j in range(len(data_x[i])):
        for k in range(len(data_x[i][j])):
            if data_x[i][j][k] == u'\xed':
                print(i, j, k)

In [9]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm = 'l2sgd',
    c2=1,
    max_iterations=100,
    all_possible_transitions=True
)

crf.fit(data_x, data_y)

/home/zeca/anaconda3/envs/ner/lib/python3.7/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='l2sgd', all_possible_transitions=True, c2=1, keep_tempfiles=None,
    max_iterations=100)

In [10]:
import joblib
joblib.dump(crf, "ner_abono.pkl")

['ner_abono.pkl']